In [ ]:
import gc
import torch

gc.collect()
torch.cuda.empty_cache()

In [1]:
import sys
sys.path.append('~/aiffel/aiffelthon')

In [2]:
!ls

checkpoint-16		modeling_encoder_decoder.py   roberta-ko-small.ipynb
configuration_utils.py	__pycache__		      runs
csv			roberta-ko-small-Copy1.ipynb  seq2seq_training_args.py
csv.zip			roberta-ko-small-Copy2.ipynb  src
import_utils.py		roberta-ko-small-Copy3.ipynb


In [1]:
!pip install rouge_score
!pip install datasets==1.0.2
# !pip install transformers

  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24955 sha256=c21dae2ef3ee96a3cc2bb4a2b43f02d05f2f2a081f0457a44d839d44cf334f71
  Stored in directory: /aiffel/.cache/pip/wheels/9b/3d/39/09558097d3119ca0a4d462df68f22c6f3c1b345ac63a09b86e
Successfully built rouge-score
     |████████████████████████████████| 1.8 MB 5.9 MB/s            
  Attempting uninstall: datasets
    Found existing installation: datasets 1.14.0
    Uninstalling datasets-1.14.0:
      Successfully uninstalled datasets-1.14.0


In [2]:
pip install transformers==4.24.0

     |████████████████████████████████| 5.5 MB 6.3 MB/s            
     |████████████████████████████████| 163 kB 84.4 MB/s            
     |████████████████████████████████| 7.6 MB 69.0 MB/s            
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.10.3
    Uninstalling tokenizers-0.10.3:
      Successfully uninstalled tokenizers-0.10.3
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.0.19
    Uninstalling huggingface-hub-0.0.19:
      Successfully uninstalled huggingface-hub-0.0.19
  Attempting uninstall: transformers
    Found existing installation: transformers 4.11.3
    Uninstalling transformers-4.11.3:
      Successfully uninstalled transformers-4.11.3
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install transformer-utils

     |████████████████████████████████| 1.7 MB 6.0 MB/s            
     |████████████████████████████████| 86 kB 11.0 MB/s            
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install packaging

Note: you may need to restart the kernel to use updated packages.


In [3]:
# 필요한 라이브러리 불러오기
import datasets
import transformers
import pandas as pd
import re
from datasets import Dataset

#Tokenizer
from transformers import BertTokenizerFast

#Encoder-Decoder Model
from transformers import EncoderDecoderModel

#Training
from seq2seq_trainer import Seq2SeqTrainer
from transformers import TrainingArguments
from seq2seq_training_args import Seq2SeqTrainingArguments
from dataclasses import dataclass, field
from typing import Optional

In [ ]:
# 경로 지정
#%cd ~/aiffel/aiffelthon

In [ ]:
#!unzip ~/aiffel/aiffelthon/csv.zip

In [3]:
#!wget https://raw.githubusercontent.com/huggingface/transformers/main/examples/legacy/seq2seq/seq2seq_trainer.py

--2022-11-07 08:23:51--  https://raw.githubusercontent.com/huggingface/transformers/main/examples/legacy/seq2seq/seq2seq_trainer.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11214 (11K) [text/plain]
Saving to: ‘seq2seq_trainer.py’

seq2seq_trainer.py  100%[===================>]  10.95K  --.-KB/s    in 0s      

2022-11-07 08:23:52 (60.1 MB/s) - ‘seq2seq_trainer.py’ saved [11214/11214]



In [ ]:
#!wget https://github.com/huggingface/transformers/blob/main/src/transformers/utils/import_utils.py

In [ ]:
#!wget https://raw.githubusercontent.com/huggingface/transformers/main/src/transformers/models/encoder_decoder/modeling_encoder_decoder.py

In [ ]:
#!wget https://raw.githubusercontent.com/huggingface/transformers/main/src/transformers/configuration_utils.py

In [7]:
# 데이터 불러오기
train_df = pd.read_csv('/aiffel/aiffel/aiffelthon/csv/train_2-3sent_Sum2.csv')
#train_df.drop(labels = 'Unnamed: 0', axis = 1, inplace = True)
train_df.rename(columns = {"input": "Text"}, inplace = True)
#train_df.rename(columns = {"sentence_onesent": "Summary"}, inplace = True)
#train_df.rename(columns = {"sentence_per_20": "Summary"}, inplace = True)
train_df.rename(columns = {"sentence_sent_2~3": "Summary"}, inplace = True)

val_df = pd.read_csv('/aiffel/aiffel/aiffelthon/csv/val_2-3sent_Sum2.csv')
#val_df.drop(labels = 'Unnamed: 0', axis = 1, inplace = True)
val_df.rename(columns = {"input": "Text"}, inplace = True)
#val_df.rename(columns = {"summary_onesent": "Summary"}, inplace = True)
#val_df.rename(columns = {"summary_per_20": "Summary"}, inplace = True)
val_df.rename(columns = {"summary_sent_2~3": "Summary"}, inplace = True)
print(len(train_df), len(val_df))

61290 9150


In [ ]:
# 10번째 row만 추출
train_df = train_df.iloc[range(0, len(train_df), 20)]
val_df = val_df.iloc[range(0, len(val_df), 10)]
print(len(train_df), len(val_df))

In [ ]:
train_df.head()

In [8]:
def preprocess_sentence(sentence):
    sentence = sentence.lower() # 텍스트 소문자화
    sentence = re.sub(r'\([^)]*\)', '', sentence) # 괄호로 닫힌 문자열 (...) 제거
    sentence = re.sub('"','', sentence) # 쌍따옴표 제거
    sentence = re.sub("'",'', sentence) # 따옴표 제거
    sentence = re.sub('\n','', sentence) # \n " 제거
    sentence = re.sub('.{2,3}\W{0,1}기자','', sentence) # 기자 이름 제거
    sentence = re.sub(r'[?.!,][/?.!,]', '', sentence) # 여러개 문장 부호를 하나의 문장부호로 바꿉니다
    sentence = re.sub("[^ㄱ-ㅎㅏ-ㅣ가-힣a-z0-9]", " ", sentence) # 영어 외 문자(숫자, 특수문자 등) 공백으로 변환
    sentence = re.sub(r'[" "]+', " ", sentence) # 여러개 공백을 하나의 공백으로 바꿉니다.
    sentence = sentence.strip() # 문장 양쪽 공백 제거

    return sentence

In [9]:
# 전체 Text 데이터에 대한 전처리 (1)
from tqdm import tqdm
clean_text = []

for s in tqdm(train_df['Text']):
    clean_text.append(preprocess_sentence(s))
    
train_df['Text'] = clean_text

100%|██████████| 61290/61290 [00:12<00:00, 5106.57it/s]


In [10]:
# 전체 headlines 데이터에 대한 전처리 
clean_headlines = []

for s in tqdm(train_df['Summary']):
      clean_headlines.append(preprocess_sentence(s))
        
train_df['Summary'] = clean_headlines

100%|██████████| 61290/61290 [00:03<00:00, 20233.11it/s]


In [11]:
# 전체 Text 데이터에 대한 전처리 (1)
from tqdm import tqdm
clean_text_val = []

for s in tqdm(val_df['Text']):
    clean_text_val.append(preprocess_sentence(s))
    
val_df['Text'] = clean_text_val

100%|██████████| 9150/9150 [00:01<00:00, 5234.19it/s]


In [12]:
# 전체 headlines 데이터에 대한 전처리 
clean_headlines_val = []

for s in tqdm(val_df['Summary']):
      clean_headlines_val.append(preprocess_sentence(s))
        
val_df['Summary'] = clean_headlines_val

100%|██████████| 9150/9150 [00:00<00:00, 20548.50it/s]


In [13]:
train_df.head()

,Text,Summary
0,내일 주말을 앞두고 있습니다 코로나19에 대한 경각심을 유지하고 생활 속 거리두기를...,코로나19에 대한 경각심을 유지하고 생활 속 거리두기를 잘 실천해 주실 것을 당부드...
1,에어컨 지침에 대해서는 저희가 관계부처하고 또 전문가들의 회의를 했는데 아직까지는 ...,구로 콜센터의 경우에는 저희가 공조시스템을 그때 의심을 했던 거는 여러 층에 걸쳐서...
2,아울러 생활방역 시행일로부터 코로나19 감염위기 상황 종료 시까지 생활방역대책본부를...,아울러 생활방역 시행일로부터 코로나19 감염위기 상황 종료 시까지 생활방역대책본부를...
3,고위험 시설에 대해서도 시설유형별로 핵심 수칙을 각각 마련하고 그 핵심 지침이 잘 ...,다음으로 5월 18일에서 19일 이틀 동안 화상으로 개최 중인 제73차 세계보건기구...
4,특별히 교육시설 종교시설 실내 체육시설 그리고 의료기관과 같이 많은 사람들이 이용하...,특별히 교육시설 종교시설 실내 체육시설 그리고 의료기관과 같이 많은 사람들이 이용하...


In [14]:
val_df.head()

,Text,Summary
0,새해가 밝았다 또 다른 한 해가 시작되었다 눈이 내린 하얀 설원이 앞에 펼쳐져 있는...,새해가 밝았다 시간이라는 미지의 설원을 걸어가면 발자국이 남을 것이다 그 발자국은 ...
1,기다리던 새해가 밝았네요 올해 제가 드디어 아빠가 됩니다 국내에서 처음으로 남의 팔...,그는 지난 2015년 공장에서 일하다 사고로 왼쪽 팔을 잃었다 그는 장애가 있던 시...
2,김정은 국무위원장이 지난달 28일부터 31일까지 열린 노동당 중앙위원회 전원회의 보...,김정은 국무위원장이 지난달 28일부터 31일까지 열린 노동당 중앙위원회 전원회의 보...
3,신 의원은 총선에 제가 다시 도전한다는 의사표시로 예비후보를 등록했다고 말했다 다른...,신 의원은 총선에 제가 다시 도전한다는 의사표시로 예비후보를 등록했다고 말했다 다른...
4,스웨덴에서는 지갑을 몸에 지니지 않아도 일상생활에 불편을 겪지 않는 이들이 있다 엄...,스웨덴에서는 지갑을 몸에 지니지 않아도 일상생활에 불편을 겪지 않는 이들이 있다 엄...


In [ ]:
# reset_index 사용
train_df.reset_index(inplace=True, drop=True)
val_df.reset_index(inplace=True, drop=True)

In [15]:
# DF > data Set으로 전환
train_data = Dataset.from_pandas(train_df) 
val_len = len(val_df) // 2
val_data = Dataset.from_pandas(val_df[:val_len])
test_data=Dataset.from_pandas(val_df[val_len:])

In [16]:
print(train_data)
print(val_data)
print(test_data)

Dataset(features: {'Text': Value(dtype='string', id=None), 'Summary': Value(dtype='string', id=None)}, num_rows: 61290)
Dataset(features: {'Text': Value(dtype='string', id=None), 'Summary': Value(dtype='string', id=None)}, num_rows: 4575)
Dataset(features: {'Text': Value(dtype='string', id=None), 'Summary': Value(dtype='string', id=None)}, num_rows: 4575)


In [14]:
val_data[0]

{'Text': '일본 사법당국의 출국금지 명령이 내려져 있던 카를로스 곤 전 르노닛산 회장이 레바논으로 비밀리에 도주했다 30일 미 일간 월스트리트저널은 소식통을 인용해 곤 전 회장이 이날 레바논 수도 베이루트 공항에 도착했다고 전했다 곤 전 회장은 보도 이후 미국의 대변인을 통해 자신이 레바논에 머물고 있다는 내용의 성명을 발표했다 이 때문에 곤 전 회장의 변호인은 물론 수사 출입국 당국까지 당혹스럽다는 반응을 보이는 상황이다 곤 전 회장은 일본의 정치적 박해 로부터 빠져나왔다고 주장하고 있다 그는 성명을 통해 유죄가 전제되고 차별이 만연하고 기본적 인권이 무시되는 잘못된 일본 사법제도의 인질 이 되지 않겠다 고 밝혔다 곤 전 회장은 르노닛산 회장 시절 회사 공금을 유용했다는 혐의 등으로 일본에서 재판을 받고 있다 지난해 4월 두 번째 보석 당시 법원에서는 거주제한 출국금지 조치를 포함한 엄격한 감시 명령을 내렸다 도쿄지방재판소는 도쿄도 미나토구 소재 단독 주택으로 곤의 주거지를 제한하고 일본에서 출국하는 것을 금지하는 등의 조건으로 보석을 인정했다 곤 전 회장의 주거지 현관에 감시 카메라를 설치해 녹화된 내용을 정기적으로 법원에 제출하며 휴대전화의 경우 인터넷 접속을 할 수 없는 1대만 변호인에게서 받아 사용하고 그 통화 이력도 법원에 제출하도록 하는 조건도 걸려 있었다 곤 전 회장의 변호인 히로나카 준이치로 변호사는 보도된 내용 이상의 것을 알지 못하며 현재 상황에 매우 놀랐다 앞으로 정보가 들어오면 법원에 제공하겠다 고 반응했다 그는 곤 전 회장의 여권은 변호사가 보관하고 있으며 변호인단이 여권을 주는 일은 있을 수 없다 고 설명했다 교도통신은 일본 출입국재류관리청의 데이터베이스에 곤 전 회장이 출국한 기록이 없는 것으로 확인됐다고 관계자의 설명을 토대로 전했다 이와 관련해 레바논 치안 당국자는 곤 전 회장으로 보이는 인물이 개인용 제트기를 이용해 베이루트에 도착했다고 nhk에 설명했다 이 인물의 입국 절차에 관해선 다른 이름으로 입국했다 카를로스 곤

In [13]:
test_data[0]

{'Text': '국토교통부장관 서승환 100 150 뭐 그쯤 이이재 위원 적어도 이게 평균시속 150 이상은 돼야 경쟁력이 있을 거라는 것을 지적하고 싶고요 그다음에 삼척 제진 간 이것도 역시 마찬가지로 평균시속 150 이상으로 돼야 될 것이라는 거고 이번에 수립하는 3차 국가철도망 구축계획에 바로 이런 부분들이 제대로 반영이 되어야 될 거다 하는 것을 말씀을 드립니다 꼭 좀 그렇게 해 주시기 바랍니다 국토교통부장관 서승환 예 공감하고 있습니다 이이재 위원 그리고 양양국제공항 문제입니다 우리 대한민국에 국제공항이 총 몇 개지요 인천 포함해서 한국공항공사 7개 총 8개입니다 그런데 양양국제공항만 현재 개항공항으로 지정되지 못하고 있습니다 그런데 양양국제공항이 금년도에 2009년 3000명에서 작년에 5만 명을 넘어섰고 올해 30만 명을 넘어설 것으로 계획되고 있습니다 여기에 개항공항이 안 됨으로 인해서 ciq 문제가 대단히 여객 불편을 초래하기 때문에 이 부분을 해결해 달라고 수차례 건의를 드렸음에도 불구하고 아직 해결되고 있지 않은 데 대해서 물론 관세청 고시와 관세법 시행령에 일치하지 않는 부분이 있기는 합니다마는 우선 당장에 이것은 4월부터 문제에 부딪히기 때문에 해결을 하실 수 있도록 장관님께서 관심을 가지시고 바로 좀 챙겨 주시기를 부탁드립니다 국토교통부장관 서승환 예 관계 부처하고 긴밀하게 협의하겠습니다',
 'Summary': '국토교통부장관 서승환 예 관계 부처하고 긴밀하게 협의하겠습니다 이이재 위원 그리고 양양국제공항 문제입니다 그런데 양양국제공항만 현재 개항공항으로 지정되지 못하고 있습니다'}

In [17]:
tokenizer = BertTokenizerFast.from_pretrained("kykim/bertshared-kor-base")

Downloading:   0%|          | 0.00/80.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/344k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.24k [00:00<?, ?B/s]

In [18]:
#tokenizer.bos_token = tokenizer.cls_token
#tokenizer.eos_token = tokenizer.sep_token
#parameter setting
batch_size=16  
encoder_max_length=256
decoder_max_length=32

In [19]:
def process_data_to_model_inputs(batch):                                                               
    # Tokenizer will automatically set [BOS] <text> [EOS]                                               
    inputs = tokenizer(batch["Text"], padding="max_length", truncation=True, max_length=encoder_max_length)
    outputs = tokenizer(batch["Summary"], padding="max_length", truncation=True, max_length=decoder_max_length)
                                                                                                        
    batch["input_ids"] = inputs.input_ids                                                               
    batch["attention_mask"] = inputs.attention_mask                                                     
    batch["decoder_input_ids"] = outputs.input_ids                                                      
    batch["labels"] = outputs.input_ids.copy()                                                          
    # mask loss for padding                                                                             
    batch["labels"] = [[-100 if token == tokenizer.pad_token_id else token for token in labels] for labels in batch["labels"]]                     
    batch["decoder_attention_mask"] = outputs.attention_mask                                                                              
                                                                                                         
    return batch  

In [20]:
#processing training data
train_data = train_data.map(
    process_data_to_model_inputs, 
    batched=True, 
    batch_size=batch_size, 
    remove_columns=["Text", "Summary"])
train_data.set_format(
    type="torch", columns=["input_ids", "attention_mask", "decoder_input_ids", "decoder_attention_mask", "labels"],)

#processing validation data
val_data = val_data.map(
    process_data_to_model_inputs, 
    batched=True, 
    batch_size=batch_size, 
    remove_columns=["Text", "Summary"])
val_data.set_format(
    type="torch", columns=["input_ids", "attention_mask", "decoder_input_ids", "decoder_attention_mask", "labels"],)

  0%|          | 0/3831 [00:00<?, ?ba/s]

  0%|          | 0/286 [00:00<?, ?ba/s]

In [21]:
# 인코더 모델 불러오기

bertshared =EncoderDecoderModel.from_pretrained("kykim/bertshared-kor-base")

Downloading:   0%|          | 0.00/589M [00:00<?, ?B/s]

The following encoder weights were not tied to the decoder ['bert/pooler']
The following encoder weights were not tied to the decoder ['bert/pooler']


In [22]:
# set special tokens
#from transformers import EncoderDecoderConfig
bertshared.config.decoder_start_token_id = tokenizer.bos_token_id                                             
bertshared.config.eos_token_id = tokenizer.eos_token_id

# sensible parameters for beam search
# set decoding params                               
bertshared.config.max_length = 32
bertshared.config.early_stopping = True
bertshared.config.no_repeat_ngram_size = 2
bertshared.config.length_penalty = 2.0
bertshared.config.num_beams = 5
bertshared.config.vocab_size = bertshared.config.encoder.vocab_size

In [23]:
# load rouge for validation
rouge = datasets.load_metric("rouge")

def compute_metrics(pred):
    labels_ids = pred.label_ids
    pred_ids = pred.predictions

    # all unnecessary tokens are removed
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    labels_ids[labels_ids == -100] = tokenizer.pad_token_id
    label_str = tokenizer.batch_decode(labels_ids, skip_special_tokens=True)

    rouge_output = rouge.compute(predictions=pred_str, references=label_str, rouge_types=["rouge2"])["rouge2"].mid

    return {
        "rouge2_precision": round(rouge_output.precision, 4),
        "rouge2_recall": round(rouge_output.recall, 4),
        "rouge2_fmeasure": round(rouge_output.fmeasure, 4),}

Downloading:   0%|          | 0.00/1.66k [00:00<?, ?B/s]

In [24]:
@dataclass
class Seq2SeqTrainingArguments(TrainingArguments):
    label_smoothing: Optional[float] = field(
        default=0.0, metadata={"help": "The label smoothing epsilon to apply (if not zero)."}
    )
    sortish_sampler: bool = field(default=False, metadata={"help": "Whether to SortishSamler or not."})
    predict_with_generate: bool = field(
        default=False, metadata={"help": "Whether to use generate to calculate generative metrics (ROUGE, BLEU)."}
    )
    adafactor: bool = field(default=False, metadata={"help": "whether to use adafactor"})
    encoder_layerdrop: Optional[float] = field(
        default=None, metadata={"help": "Encoder layer dropout probability. Goes into model.config."}
    )
    decoder_layerdrop: Optional[float] = field(
        default=None, metadata={"help": "Decoder layer dropout probability. Goes into model.config."}
    )
    dropout: Optional[float] = field(default=None, metadata={"help": "Dropout probability. Goes into model.config."})
    attention_dropout: Optional[float] = field(
        default=None, metadata={"help": "Attention dropout probability. Goes into model.config."}
    )
    lr_scheduler: Optional[str] = field(
        default="linear", metadata={"help": f"Which lr scheduler to use."}
    )

In [28]:
training_args = Seq2SeqTrainingArguments(
    output_dir="~/aiffel/aiffelthon/ch",
    num_train_epochs = 6,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    predict_with_generate=True,
    do_train=True,
    do_eval=True,
    logging_steps=4, 
    save_steps=16, 
    eval_steps=7500, 
    warmup_steps=3000,
    max_steps=16,
    overwrite_output_dir=True,
    save_total_limit=1,
    fp16=True,)

# instantiate trainer
trainer = Seq2SeqTrainer(
    model=bertshared,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_data,
    eval_dataset=val_data,)
trainer.train()

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
max_steps is given, it will override any value given in num_train_epochs
Using cuda_amp half precision backend
/opt/conda/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 61290
  Num Epochs = 1
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 16
  Number of trainable parameters = 

Step,Training Loss
4,0.067100
8,0.057200
12,0.082100
16,0.076000


Saving model checkpoint to /aiffel/aiffel/aiffelthon/ch/checkpoint-16
Configuration saved in /aiffel/aiffel/aiffelthon/ch/checkpoint-16/config.json
Model weights saved in /aiffel/aiffel/aiffelthon/ch/checkpoint-16/pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=16, training_loss=0.07062062248587608, metrics={'train_runtime': 29.0873, 'train_samples_per_second': 17.602, 'train_steps_per_second': 0.55, 'total_flos': 90162516197376.0, 'train_loss': 0.07062062248587608, 'epoch': 0.01})

In [30]:
tokenizer = BertTokenizerFast.from_pretrained("kykim/bertshared-kor-base")
model = EncoderDecoderModel.from_pretrained('/aiffel/aiffel/aiffelthon/ch/checkpoint-16/').to("cuda")
batch_size = 32

# map data correctly
def generate_summary(batch):
    # Tokenizer will automatically set [BOS] <text> [EOS]
    inputs = tokenizer(batch["Text"], padding="max_length", truncation=True, max_length=32, return_tensors="pt")
    input_ids = inputs.input_ids.to("cuda")
    attention_mask = inputs.attention_mask.to("cuda")
    outputs = model.generate(input_ids, attention_mask=attention_mask)
    # all special tokens including will be removed
    output_str = tokenizer.batch_decode(outputs, skip_special_tokens=True)

    batch["pred"] = output_str

    return batch
results = test_data.map(generate_summary, batched=True, batch_size=batch_size, remove_columns=["Text"])
pred_str = results["pred"]
label_str = results["Summary"]

loading file vocab.txt from cache at /aiffel/.cache/huggingface/hub/models--kykim--bertshared-kor-base/snapshots/8ea27677d006a547aee2a7753d1ff18a0346860c/vocab.txt
loading file tokenizer.json from cache at None
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at None
loading file tokenizer_config.json from cache at /aiffel/.cache/huggingface/hub/models--kykim--bertshared-kor-base/snapshots/8ea27677d006a547aee2a7753d1ff18a0346860c/tokenizer_config.json
loading configuration file config.json from cache at /aiffel/.cache/huggingface/hub/models--kykim--bertshared-kor-base/snapshots/8ea27677d006a547aee2a7753d1ff18a0346860c/config.json
Model config EncoderDecoderConfig {
  "_commit_hash": "8ea27677d006a547aee2a7753d1ff18a0346860c",
  "_name_or_path": "kykim/bertshared-kor-base",
  "architectures": [
    "EncoderDecoderModel"
  ],
  "decoder": {
    "_name_or_path": "",
    "add_cross_attention": true,
    "architectures": [
      "BertForMaske

  0%|          | 0/143 [00:00<?, ?ba/s]

Setting `pad_token_id` to `eos_token_id`:3 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:3 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:3 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:3 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:3 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:3 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:3 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:3 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:3 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:3 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:3 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:3 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:3 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:3 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:3 for o

In [31]:
for num in range(10):
    print('predicted sentence : ',pred_str[num])
    print('real sentence : ', label_str[num])
    print('-'*100)

predicted sentence :  8 8분분 시간 시간 제약제약제약 때문에 때문에 인해 인해 계속 계속 연관 연관 연결된된된 문제 문제 문제로 문제로 문제가 문제가문제가문제가 문제가 계속
real sentence :  이군현 위원 그런데 아까 말씀드렸듯이 이게 14조면 국가 전체 예산의 3 6 에 불과하다고요 이렇게 농림부 예산이 증가하고 있지 않은데 내년에 1500억 쌀생산조정제 예산을 확실하게 확보하겠다 하는 것에 대해서 많은 농민단체나 국민들이 의심을 하고 있다고요 과연 농림부가 그런 능력이 있는가 또 그런 의지가 있는가 하는 것에 대해서 의심을 하고 있거든요
----------------------------------------------------------------------------------------------------
predicted sentence :  농림 농림축축산산식품식품부장부장부부총리총리자자선선선을선을 선 선 동 동감감을감을도를도를도로도로
real sentence :  김철민 위원 용산 화상경마장에 대해서 잠시 질문을 하도록 하겠습니다 당초 마사회의 화상경마장 이전 과정에서 인근 학교와의 거리 또 민원 발생 가능성 등을 우리 농림부에 허위로 보고하고 이에 대해 제대로 검증하지 않고 이전 승인을 하면서 문제가 촉발되었다는 지적이 있습니다 후보자께서는 장관에 취임하시면 당시의 이전 승인 과정을 면밀히 재검토하고 그 내용에 대해서 국회에 보고할 용의가 있으십니까
----------------------------------------------------------------------------------------------------
predicted sentence :  농림 농림축축산식품식품부장부장부부후보후보자자 김영 김영록록 위원 위원위원은위원은위원위원님님님이님이 말씀 말씀
real sentence :  농림축산식품부장관후보자 김영록 4대강 사업에 대한 평가는 다양한 평가가 있습니다 저는 4대강 사업에 대해서 당초

In [27]:
for num in range(10):
    print('predicted sentence : ',pred_str[num])
    print('real sentence : ', label_str[num])
    print('-'*120)

predicted sentence :  올해 올해 국가 국가 전체 전체 예산 예산 증가 증가비율비율 비율이 비율이 비율을 비율을 비율 비율비율 비율을 정도를 정도를 정도 정도정도정도 정도 정도로 정도로 정도만 정도만
real sentence :  이군현 위원 그런데 아까 말씀드렸듯이 이게 14조면 국가 전체 예산의 3 6 에 불과하다고요 이렇게 농림부 예산이 증가하고 있지 않은데 내년에 1500억 쌀생산조정제 예산을 확실하게 확보하겠다 하는 것에 대해서 많은 농민단체나 국민들이 의심을 하고 있다고요 과연 농림부가 그런 능력이 있는가 또 그런 의지가 있는가 하는 것에 대해서 의심을 하고 있거든요
------------------------------------------------------------------------------------------------------------------------
predicted sentence :  농림 농림축축산식품식품부장부장부부 부 부부감감감을감을 하기 하기 하기에 하기에하기에하기에 있기에 있기에 있기 있기 있기에 때문에 때문에
real sentence :  김철민 위원 용산 화상경마장에 대해서 잠시 질문을 하도록 하겠습니다 당초 마사회의 화상경마장 이전 과정에서 인근 학교와의 거리 또 민원 발생 가능성 등을 우리 농림부에 허위로 보고하고 이에 대해 제대로 검증하지 않고 이전 승인을 하면서 문제가 촉발되었다는 지적이 있습니다 후보자께서는 장관에 취임하시면 당시의 이전 승인 과정을 면밀히 재검토하고 그 내용에 대해서 국회에 보고할 용의가 있으십니까
------------------------------------------------------------------------------------------------------------------------
predicted sentence :  농림 농림축축산식품식품부장부장부부제제 제 제션이션이션을션을 대폭 대폭 늘어나 늘어나지지 않으면 않으면 안 안 된다고

In [ ]:
print("ROUGE 1 SCORE: ",rouge.compute(predictions=pred_str, references=label_str, rouge_types=["rouge1"])["rouge1"].mid)
print("ROUGE 2 SCORE: ",rouge.compute(predictions=pred_str, references=label_str, rouge_types=["rouge2"])["rouge2"].mid)
print("ROUGE L SCORE: ",rouge.compute(predictions=pred_str, references=label_str, rouge_types=["rougeL"])["rougeL"].mid)